In [5]:
# --- CELDA 1 CORREGIDA: INSTALACIÓN CON PARCHE ---
# Añadimos 'protobuf' para arreglar el error rojo
!pip install -U -q transformers datasets accelerate protobuf==3.20.*

import os
os.environ["WANDB_DISABLED"] = "true"
print("✅ Entorno listo y LIBRERÍAS PARCHEADAS.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 4.3 MB/s eta 0:00:00a 0:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
opentelemetry-proto 1.37.0 requires protobuf<7.0,>=5.0, but you have protobuf 3.20.3 which is incompatible.
onnx 1.18.0 requires protobuf>=4.25.1, but you have protobuf 3.20.3 which is incompatible.
a2a-sdk 0.3.10 requires protobuf>=5.29.5, but you have protobuf 3.20.3 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
tensorflow-metadata 1.17.2 requires protobuf>=4.25.2; python_version >= "3.11", but you have protobuf 3.20.3 which is incompatible.
pydrive2 1.21.3 requires cryptograp

In [6]:
import os

dataset_path = None
model_path = None

print("🔍 Escaneando archivos...")

for root, dirs, files in os.walk("/kaggle/input"):
    # 1. Buscar el Dataset Limpio
    for file in files:
        if "CLEANED.txt" in file:
            dataset_path = os.path.join(root, file)
            print(f"   📄 Dataset Limpio encontrado: {dataset_path}")

    # 2. Buscar el Modelo (Buscamos config.json)
    for file in files:
        if file == "config.json":
            # Si encontramos config.json, esta es la carpeta raíz del modelo
            model_path = root
            print(f"   🧠 Modelo V5 encontrado en: {model_path}")

if not dataset_path or not model_path:
    print("❌ ERROR CRÍTICO: No encuentro los archivos. Revisa los inputs.")
else:
    print("✅ Todo localizado. Podemos proceder.")

🔍 Escaneando archivos...
   📄 Dataset Limpio encontrado: /kaggle/input/dataset-v3-cleaned/dataset_FINAL_V3_CLEANED.txt
   🧠 Modelo V5 encontrado en: /kaggle/input/ascii-art-v5/pytorch/hardcore-training/1
✅ Todo localizado. Podemos proceder.


In [7]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

print(f"⚙️ Cargando modelo desde {model_path}...")
try:
    tokenizer = GPT2Tokenizer.from_pretrained(model_path)
    model = GPT2LMHeadModel.from_pretrained(model_path)
    tokenizer.pad_token = tokenizer.eos_token
    print("✅ Modelo V5 cargado en memoria.")
except Exception as e:
    print(f"❌ Error al cargar: {e}")

⚙️ Cargando modelo desde /kaggle/input/ascii-art-v5/pytorch/hardcore-training/1...
✅ Modelo V5 cargado en memoria.


In [8]:
import shutil
from transformers import TextDataset, DataCollatorForLanguageModeling

# Copiar a /kaggle/working
local_dataset = "/kaggle/working/training_data_clean.txt"
print("🚚 Copiando dataset a zona de trabajo...")
shutil.copy(dataset_path, local_dataset)

print("📚 Procesando Dataset (Tokenizando)...")
train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path=local_dataset,
    block_size=128,
    overwrite_cache=True
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False
)

print(f"✅ Datos listos. Bloques de entrenamiento: {len(train_dataset)}")

🚚 Copiando dataset a zona de trabajo...
📚 Procesando Dataset (Tokenizando)...
✅ Datos listos. Bloques de entrenamiento: 42631


In [9]:
from transformers import Trainer, TrainingArguments

output_dir = "/kaggle/working/ascii-gpt-v6-clean"

training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    
    # --- CONFIGURACIÓN DE TIEMPO ---
    num_train_epochs=25,           # 25 Épocas (Calculado para ~5-6 horas)
    per_device_train_batch_size=8, # Batch size estable
    learning_rate=1e-5,            # Velocidad baja para alta precisión
    
    # --- SEGURIDAD ---
    save_steps=2000,               # Guardar checkpoint cada ~30 mins
    save_total_limit=3,            # Guardar los últimos 3 checkpoints (por si acaso)
    
    # --- TÉCNICO ---
    fp16=True,                     # Aceleración GPU
    report_to="none",
    logging_steps=100
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

print("⏱️ Configuración lista para entrenamiento de larga duración.")

⏱️ Configuración lista para entrenamiento de larga duración.


In [ ]:
print("🚀 INICIANDO ENTRENAMIENTO DE V6 (THE CLEAN ONE)...")
print("   Nota: Esto tardará varias horas. Asegúrate de que la pestaña no se duerma.")
trainer.train()
print("🏁 ¡META ALCANZADA!")

🚀 INICIANDO ENTRENAMIENTO DE V6 (THE CLEAN ONE)...
   Nota: Esto tardará varias horas. Asegúrate de que la pestaña no se duerma.


`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
100,1.131000
200,1.105700
300,1.105700
400,1.157700
500,1.122000
600,1.103000
700,1.097700
800,1.113400
900,1.114400
1000,1.101900


We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
You may ignore this warning if your `pad_token_id` (50256) is identical to the `bos_token_id` (50256), `eos_token_id` (50256), or the `sep_token_id` (None), and your input is not padded.


In [ ]:
import shutil

final_path = "/kaggle/working/modelo_final_v6_clean"
zip_name = "/kaggle/working/ASCII-GPT-V6-CLEAN"

print("💾 Guardando el modelo definitivo...")
trainer.save_model(final_path)
tokenizer.save_pretrained(final_path)

print("📦 Comprimiendo...")
shutil.make_archive(zip_name, 'zip', final_path)

print(f"🎉 LISTO. Descarga {zip_name}.zip")

In [12]:
from IPython.display import FileLink
FileLink(r'ASCII-GPT-V6-CLEAN.zip')


/kaggle/working/ASCII-GPT-V6-CLEAN.zip